### Brainstorming
Given features:
- players
- outcome (1)
- teams?
- lineup score

Player lineups give different scores -> used as an input

https://sportsdata.io/developers/fantasy-scoring-system/nba

### Import Libraries and Data

In [64]:
import pandas as pd
import glob
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [52]:
filepaths = glob.glob("matchups-*.csv")
data = pd.concat([pd.read_csv(file) for file in filepaths], ignore_index=True)

In [53]:
x_columns = [
    'home_team', 'away_team', 'season', 'starting_min',
    'home_0','home_1','home_2','home_3','home_4',
    'away_0','away_1','away_2','away_3','away_4',
    'outcome'
]
X = data.filter(items=x_columns, axis=1)

In [73]:
encoder = LabelEncoder()
for column in x_columns:
    X[column] = encoder.fit_transform(X[column])
print(X)

        home_team  away_team  season  starting_min  home_0  home_1  home_2  \
0              14         27       0             0      51     556     527   
1              14         27       0             5      51     556     527   
2              14         27       0             7     555     587     576   
3              14         27       0             9     555     587     576   
4              14         27       0            10     575     633     719   
...           ...        ...     ...           ...     ...     ...     ...   
236907         23         26       8            34     437     714     702   
236908         23         26       8            38     437     560     673   
236909         23         26       8            39     437     560     673   
236910         23         26       8            41      46     414     496   
236911         23         26       8            44      46     414     496   

        home_3  home_4  away_0  away_1  away_2  away_3  away_4 

In [55]:
clf0 = RandomForestClassifier(max_depth=10, random_state=0)
clf1 = RandomForestClassifier(max_depth=10, random_state=0)
clf2 = RandomForestClassifier(max_depth=10, random_state=0)
clf3 = RandomForestClassifier(max_depth=10, random_state=0)
clf4 = RandomForestClassifier(max_depth=10, random_state=0)

Y0 = data['home_0']
Y1 = data['home_1']
Y2 = data['home_2']
Y3 = data['home_3']
Y4 = data['home_4']

X0 = X.drop(columns=['home_0'])
X1 = X.drop(columns=['home_1'])
X2 = X.drop(columns=['home_2'])
X3 = X.drop(columns=['home_3'])
X4 = X.drop(columns=['home_4'])

### Train Classifiers

In [56]:
clf0.fit(X0, Y0)

RandomForestClassifier(max_depth=10, random_state=0)

In [66]:
_ = joblib.dump(clf0, 'models/clf0.joblib.pkl', compress=9)

In [57]:
clf1.fit(X1, Y1)

RandomForestClassifier(max_depth=10, random_state=0)

In [67]:
_ = joblib.dump(clf1, 'models/clf1.joblib.pkl', compress=9)

In [58]:
clf2.fit(X2, Y2)

RandomForestClassifier(max_depth=10, random_state=0)

In [68]:
_ = joblib.dump(clf2, 'models/clf2.joblib.pkl', compress=9)

In [59]:
clf3.fit(X3, Y3)

RandomForestClassifier(max_depth=10, random_state=0)

In [69]:
_ = joblib.dump(clf3, 'models/clf3.joblib.pkl', compress=9)

In [60]:
clf4.fit(X4, Y4)

RandomForestClassifier(max_depth=10, random_state=0)

In [70]:
_ = joblib.dump(clf4, 'models/clf4.joblib.pkl', compress=9)

### Evaluate with Classifiers

In [72]:
#Import test data
test_data = pd.read_csv(f"NBA_test.csv")

#Apply label encoding to columns
for column in x_columns:
    if column == 'outcome':
        continue
    test_data[column] = encoder.transform(test_data[column])

#Add "outcome" column with value 1 (assume winning value)
test_data['outcome'] = '1'

ValueError: invalid literal for int() with base 10: 'IND'

In [ ]:
#Import models
clf0 = joblib.load('models/clf0.pkl')
clf1 = joblib.load('models/clf1.pkl')
clf2 = joblib.load('models/clf2.pkl')
clf3 = joblib.load('models/clf3.pkl')
clf4 = joblib.load('models/clf4.pkl')

In [36]:
def evaluate(row):
    if row['home_0'] == encoder:
        return clf0.predict_proba([row.drop(labels='home_0')])
    elif row['home_1'] == '?':
        return clf1.predict_proba([row.drop(labels='home_1')])
    elif row['home_2'] == '?':
        return clf2.predict_proba([row.drop(labels='home_2')])
    elif row['home_3'] == '?':
        return clf3.predict_proba([row.drop(labels='home_3')])
    elif row['home_4'] == '?':
        return clf4.predict_proba([row.drop(labels='home_4')])
    else:
        print(row)
        raise Exception("Invalid row provided.")

In [37]:
predictions = test_data.apply(evaluate, axis=1)
print(predictions)

season            0
home_team        11
away_team         1
starting_min     16
home_0          137
home_1           83
home_2          176
home_3          163
home_4            0
away_0           19
away_1          174
away_2          216
away_3          300
away_4          179
outcome           1
Name: 0, dtype: object


Exception: Invalid row provided.